<a href="https://colab.research.google.com/github/dlwub/Final_Project_AIMS/blob/main/Final_Project_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part I: Loading Data and Preprocessing

### Loading Data

In [ ]:
!unzip /content/Project_data-20240529T124805Z-001.zip

unzip:  cannot find or open /content/Project_data-20240529T124805Z-001.zip, /content/Project_data-20240529T124805Z-001.zip.zip or /content/Project_data-20240529T124805Z-001.zip.ZIP.


### We use Consumer Price Index (CPI) data with 10 other macroeconomic indicators (Real GDP, Money Supply, Retail Sales Volume, Exchange Rate, Federal Government Current Expenditure, Trade Balance, Interest Rate, Producer Price Index: Manufacturing, and Producer Price Index:Industry).  

### Importing Pandas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from scipy import stats

### Converting exchange rate data from daily to monthly using pandas resample method

In [ ]:
exchange_rate = pd.read_csv("/content/Project_data/Exchange_rate.csv")

In [ ]:
exchange_rate.head()

,DATE,DEXSFUS
0,1/2/1980,0.8271
1,1/3/1980,0.8271
2,1/4/1980,0.8268
3,1/7/1980,0.8268
4,1/8/1980,0.8268


In [ ]:
exchange_rate["DATE"] = pd.to_datetime(exchange_rate["DATE"])
exchange_rate.set_index('DATE', inplace=True)
exchange_rate = exchange_rate.asfreq('M').ffill()

### some rows contain '.' instead of numeric values.

In [ ]:
problematic_indices = exchange_rate.index[exchange_rate['DEXSFUS'] == '.']

In [ ]:
problematic_indices

DatetimeIndex(['1982-05-31', '1993-05-31', '1999-05-31', '2004-05-31',
               '2010-05-31', '2010-12-31', '2021-05-31', '2021-12-31'],
              dtype='datetime64[ns]', name='DATE', freq=None)

### We replace '.' values with the average of the row before and after

In [ ]:
for idx in problematic_indices:
  prev_idx = exchange_rate.index[exchange_rate.index < idx][-1]
  next_idx = exchange_rate.index[exchange_rate.index > idx][0]
  avg_value = (float(exchange_rate.loc[prev_idx]['DEXSFUS']) + float(exchange_rate.loc[next_idx]['DEXSFUS']))/2
  exchange_rate.at[idx, 'DEXSFUS'] = avg_value

### The 'DEXSFUS' column is string object. Hence, we convert it to float to apply z-score.

In [ ]:
exchange_rate['DEXSFUS'] = pd.to_numeric(exchange_rate['DEXSFUS'], errors='coerce')

In [ ]:
exchange_rate.rename(columns={'DEXSFUS': 'EXC'}, inplace=True)

In [ ]:
exchange_rate.to_csv('ex_rate.csv')

In [ ]:
ex_rate = pd.read_csv("/content/ex_rate.csv")

In [ ]:
ex_rate

,DATE,EXC
0,1980-01-31,0.8172
1,1980-02-29,0.8079
2,1980-03-31,0.8102
3,1980-04-30,0.7999
4,1980-05-31,0.7999
...,...,...
526,2023-11-30,18.8698
527,2023-12-31,18.8698
528,2024-01-31,18.6527
529,2024-02-29,19.1884


### Change the date format from '%Y-%M-%D' to '%Y-%M'

In [ ]:
ex_rate['DATE'] = pd.to_datetime(ex_rate['DATE']).dt.strftime('%Y-%m')

In [ ]:
ex_rate.to_csv('ex_rate.csv', index=False)

In [ ]:
ex_rate = pd.read_csv("/content/ex_rate.csv")

In [ ]:
ex_rate

,DATE,EXC
0,1980-01,0.8172
1,1980-02,0.8079
2,1980-03,0.8102
3,1980-04,0.7999
4,1980-05,0.7999
...,...,...
526,2023-11,18.8698
527,2023-12,18.8698
528,2024-01,18.6527
529,2024-02,19.1884


### Convert interest rate data from daily to monthly

In [ ]:
interest_rate = pd.read_csv("/content/Project_data/Interest_rate.csv")

In [ ]:
interest_rate.head()

,DATE,INTGSTZAM193N
0,1/1/1957,3.25
1,2/1/1957,3.25
2,3/1/1957,3.25
3,4/1/1957,3.25
4,5/1/1957,3.25


In [ ]:
interest_rate["DATE"] = pd.to_datetime(interest_rate["DATE"])
interest_rate.set_index('DATE', inplace=True)

In [ ]:
interest_rate['INTGSTZAM193N'] = pd.to_numeric(interest_rate['INTGSTZAM193N'], errors='coerce')

In [ ]:
interest_rate

,INTGSTZAM193N
DATE,
1957-01-01,3.25
1957-02-01,3.25
1957-03-01,3.25
1957-04-01,3.25
1957-05-01,3.25
...,...
2023-09-01,8.33
2023-10-01,8.55
2023-11-01,8.52


In [ ]:
interest_rate.to_csv('int_rate.csv')

### Change the date format from '%Y-%M-%D' to '%Y-%M'

In [ ]:
interest_rate = pd.read_csv("/content/int_rate.csv")

In [ ]:
interest_rate['DATE'] = pd.to_datetime(interest_rate['DATE']).dt.strftime('%Y-%m')

In [ ]:
interest_rate

,DATE,INTGSTZAM193N
0,1957-01,3.25
1,1957-02,3.25
2,1957-03,3.25
3,1957-04,3.25
4,1957-05,3.25
...,...,...
800,2023-09,8.33
801,2023-10,8.55
802,2023-11,8.52
803,2023-12,8.45


In [ ]:
interest_rate.rename(columns={'INTGSTZAM193N': 'INT'}, inplace=True)

In [ ]:
interest_rate.to_csv('int_rate.csv', index=False)

In [ ]:
int_rate = pd.read_csv("/content/int_rate.csv")

In [ ]:
int_rate

,DATE,INT
0,1957-01,3.25
1,1957-02,3.25
2,1957-03,3.25
3,1957-04,3.25
4,1957-05,3.25
...,...,...
800,2023-09,8.33
801,2023-10,8.55
802,2023-11,8.52
803,2023-12,8.45


### Money supply data (It is monthly data)

In [ ]:
Money_supply = pd.read_csv("/content/Project_data/Money_supply.csv")

In [ ]:
Money_supply

,DATE,MYAGM2ZAM189N
0,3/1/1965,2.886850e+09
1,4/1/1965,2.949080e+09
2,5/1/1965,2.954000e+09
3,6/1/1965,2.975480e+09
4,7/1/1965,3.018010e+09
...,...,...
622,1/1/2017,2.591870e+12
623,2/1/2017,2.555400e+12
624,3/1/2017,2.584130e+12
625,4/1/2017,2.574110e+12


In [ ]:
Money_supply["DATE"] = pd.to_datetime(Money_supply["DATE"])
Money_supply.set_index('DATE', inplace=True)

In [ ]:
Money_supply.head()

,MYAGM2ZAM189N
DATE,
1965-03-01,2.886850e+09
1965-04-01,2.949080e+09
1965-05-01,2.954000e+09
1965-06-01,2.975480e+09
1965-07-01,3.018010e+09


In [ ]:
Money_supply['MYAGM2ZAM189N'] = pd.to_numeric(Money_supply['MYAGM2ZAM189N'], errors='coerce')

In [ ]:
Money_supply

,MYAGM2ZAM189N
DATE,
1965-03-01,2.886850e+09
1965-04-01,2.949080e+09
1965-05-01,2.954000e+09
1965-06-01,2.975480e+09
1965-07-01,3.018010e+09
...,...
2017-01-01,2.591870e+12
2017-02-01,2.555400e+12
2017-03-01,2.584130e+12


In [ ]:
Money_supply.to_csv('Money_supply.csv')

### Converting the date format from '%Y-%M-%D' to '%Y-%M'

In [ ]:
Money_supply = pd.read_csv("/content/Money_supply.csv")

In [ ]:
Money_supply['DATE'] = pd.to_datetime(Money_supply['DATE']).dt.strftime('%Y-%m')

In [ ]:
Money_supply.rename(columns={'MYAGM2ZAM189N': 'M2'}, inplace=True)

In [ ]:
Money_supply.to_csv('M2.csv', index=False)

In [ ]:
M2 = pd.read_csv("/content/M2.csv")

In [ ]:
M2

,DATE,M2
0,1965-03,2.886850e+09
1,1965-04,2.949080e+09
2,1965-05,2.954000e+09
3,1965-06,2.975480e+09
4,1965-07,3.018010e+09
...,...,...
622,2017-01,2.591870e+12
623,2017-02,2.555400e+12
624,2017-03,2.584130e+12
625,2017-04,2.574110e+12


### Converting PPI_Industrial data from quarterly to monthly using interpolation

In [ ]:
PPI_Industrial = pd.read_csv("/content/Project_data/PPI_Industrial.csv", converters={'DATE': pd.to_datetime})

In [ ]:
# Is quarterly data
PPI_Industrial.head(10)

,DATE,ZAFPIEATI02GYQ
0,1971-01-01,2.742361
1,1971-04-01,4.099550
2,1971-07-01,4.068459
3,1971-10-01,2.822102
4,1972-01-01,3.783177
5,1972-04-01,4.146657
6,1972-07-01,7.998358
7,1972-10-01,10.075090
8,1973-01-01,9.844416
9,1973-04-01,10.408369


In [ ]:
PPI_Industrial['DATE'] = pd.to_datetime(PPI_Industrial['DATE']).dt.to_period('M')

In [ ]:
PPI_Industrial = PPI_Industrial.set_index('DATE').resample('M').interpolate()

In [ ]:
PPI_Industrial

,ZAFPIEATI02GYQ
DATE,
1971-01,2.742361
1971-02,3.194757
1971-03,3.647154
1971-04,4.099550
1971-05,4.089186
...,...
2022-06,16.503959
2022-07,17.121706
2022-08,17.485309


In [ ]:
PPI_Industrial.rename(columns={'ZAFPIEATI02GYQ': 'PPI_IND'}, inplace=True)

In [ ]:
PPI_Industrial.to_csv('PPI_Industrial.csv')

In [ ]:
PPI_Industrial = pd.read_csv("/content/PPI_Industrial.csv")

In [ ]:
PPI_Industrial

,DATE,PPI_IND
0,1971-01,2.742361
1,1971-02,3.194757
2,1971-03,3.647154
3,1971-04,4.099550
4,1971-05,4.089186
...,...,...
617,2022-06,16.503959
618,2022-07,17.121706
619,2022-08,17.485309
620,2022-09,17.848912


### PPI_Manufacturing (monthly data)

In [ ]:
PPI_Manufacturing = pd.read_csv("/content/Project_data/PPI_Manufacturing.csv", converters={'DATE': pd.to_datetime})

In [ ]:
PPI_Manufacturing.head()

,DATE,ZAFPPDMMINMEI
0,1970-01-01,1.625744
1,1970-02-01,1.625744
2,1970-03-01,1.625744
3,1970-04-01,1.625744
4,1970-05-01,1.625744


In [ ]:
PPI_Manufacturing["DATE"] = pd.to_datetime(PPI_Manufacturing["DATE"]).dt.to_period('M')

In [ ]:
PPI_Manufacturing = PPI_Manufacturing.set_index('DATE').resample('M').interpolate()

In [ ]:
PPI_Manufacturing.head()

,ZAFPPDMMINMEI
DATE,
1970-01,1.625744
1970-02,1.625744
1970-03,1.625744
1970-04,1.625744
1970-05,1.625744


In [ ]:
PPI_Manufacturing

,ZAFPPDMMINMEI
DATE,
1970-01,1.625744
1970-02,1.625744
1970-03,1.625744
1970-04,1.625744
1970-05,1.625744
...,...
2022-08,159.862262
2022-09,161.024427
2022-10,161.670074


In [ ]:
PPI_Manufacturing.rename(columns={'ZAFPPDMMINMEI': 'PPI_MNG'}, inplace=True)

In [ ]:
PPI_Manufacturing.to_csv('PPI_Manufacturing.csv')

In [ ]:
PPI_Manufacturing = pd.read_csv("/content/PPI_Manufacturing.csv")

In [ ]:
PPI_Manufacturing

,DATE,PPI_MNG
0,1970-01,1.625744
1,1970-02,1.625744
2,1970-03,1.625744
3,1970-04,1.625744
4,1970-05,1.625744
...,...,...
631,2022-08,159.862262
632,2022-09,161.024427
633,2022-10,161.670074
634,2022-11,162.444851


### Converting GDP data from quarterly to monthly using interpolation

In [ ]:
Real_GDP = pd.read_csv("/content/Project_data/Real_GDP.csv")

In [ ]:
# Is quarterly data
Real_GDP.head()

,DATE,NGDPRSAXDCZAQ
0,1/1/1993,570937.3
1,4/1/1993,574835.4
2,7/1/1993,582207.0
3,10/1/1993,587176.1
4,1/1/1994,586899.4


In [ ]:
Real_GDP['DATE'] = pd.to_datetime(Real_GDP['DATE']).dt.to_period('M')
Real_GDP = Real_GDP.set_index('DATE').resample('M').interpolate()

In [ ]:
Real_GDP

,NGDPRSAXDCZAQ
DATE,
1993-01,5.709373e+05
1993-02,5.722367e+05
1993-03,5.735360e+05
1993-04,5.748354e+05
1993-05,5.772926e+05
...,...
2023-03,1.156183e+06
2023-04,1.157941e+06
2023-05,1.156985e+06


In [ ]:
Real_GDP.rename(columns={'NGDPRSAXDCZAQ': 'GDP'}, inplace=True)

In [ ]:
Real_GDP.to_csv('Real_GDP.csv')

In [ ]:
Real_GDP = pd.read_csv("/content/Real_GDP.csv")

In [ ]:
Real_GDP

,DATE,GDP
0,1993-01,5.709373e+05
1,1993-02,5.722367e+05
2,1993-03,5.735360e+05
3,1993-04,5.748354e+05
4,1993-05,5.772926e+05
...,...,...
362,2023-03,1.156183e+06
363,2023-04,1.157941e+06
364,2023-05,1.156985e+06
365,2023-06,1.156030e+06


### Retail Sales Volume (monthly data)

In [ ]:
RS_volume = pd.read_csv("/content/Project_data/Retail_sales_volume.csv")

In [ ]:
RS_volume.head()

,DATE,ZAFSLRTTO01MLM
0,1/1/1977,3.844167e+10
1,2/1/1977,3.744667e+10
2,3/1/1977,4.099052e+10
3,4/1/1977,3.937761e+10
4,5/1/1977,3.888693e+10


In [ ]:
RS_volume["DATE"] = pd.to_datetime(RS_volume["DATE"]).dt.to_period('M')
RS_volume.set_index('DATE', inplace=True)

In [ ]:
RS_volume.head()

,ZAFSLRTTO01MLM
DATE,
1977-01,3.844167e+10
1977-02,3.744667e+10
1977-03,4.099052e+10
1977-04,3.937761e+10
1977-05,3.888693e+10


In [ ]:
RS_volume

,ZAFSLRTTO01MLM
DATE,
1977-01,3.844167e+10
1977-02,3.744667e+10
1977-03,4.099052e+10
1977-04,3.937761e+10
1977-05,3.888693e+10
...,...
2023-06,9.111300e+10
2023-07,8.999000e+10
2023-08,9.220800e+10


In [ ]:
RS_volume.rename(columns={'ZAFSLRTTO01MLM': 'RSV'}, inplace=True)

In [ ]:
RS_volume.to_csv('RS_volume.csv')

In [ ]:
RS_volume = pd.read_csv("/content/RS_volume.csv")

In [ ]:
RS_volume

,DATE,RSV
0,1977-01,3.844167e+10
1,1977-02,3.744667e+10
2,1977-03,4.099052e+10
3,1977-04,3.937761e+10
4,1977-05,3.888693e+10
...,...,...
557,2023-06,9.111300e+10
558,2023-07,8.999000e+10
559,2023-08,9.220800e+10
560,2023-09,9.262400e+10


### Trade Balance (Monthly data)

In [ ]:
trade_balance = pd.read_csv("/content/Project_data/Trade_balance.csv")

In [ ]:
trade_balance.head()

,DATE,XTNTVA01ZAM664S
0,1/1/1957,-12924310.0
1,2/1/1957,-17258810.0
2,3/1/1957,-11865710.0
3,4/1/1957,-24045120.0
4,5/1/1957,-15366720.0


In [ ]:
trade_balance["DATE"] = pd.to_datetime(trade_balance["DATE"]).dt.to_period('M')
trade_balance.set_index('DATE', inplace=True)

In [ ]:
trade_balance.head()

,XTNTVA01ZAM664S
DATE,
1957-01,-12924310.0
1957-02,-17258810.0
1957-03,-11865710.0
1957-04,-24045120.0
1957-05,-15366720.0


In [ ]:
trade_balance

,XTNTVA01ZAM664S
DATE,
1957-01,-1.292431e+07
1957-02,-1.725881e+07
1957-03,-1.186571e+07
1957-04,-2.404512e+07
1957-05,-1.536672e+07
...,...
2023-10,6.120901e+08
2023-11,1.872539e+10
2023-12,2.862267e+09


In [ ]:
trade_balance.rename(columns={'XTNTVA01ZAM664S': 'Tr_bal'}, inplace=True)

In [ ]:
trade_balance.to_csv('trade_balance.csv')

In [ ]:
trade_balance = pd.read_csv("/content/trade_balance.csv")

In [ ]:
trade_balance

,DATE,Tr_bal
0,1957-01,-1.292431e+07
1,1957-02,-1.725881e+07
2,1957-03,-1.186571e+07
3,1957-04,-2.404512e+07
4,1957-05,-1.536672e+07
...,...,...
801,2023-10,6.120901e+08
802,2023-11,1.872539e+10
803,2023-12,2.862267e+09
804,2024-01,1.771628e+10


### Federal Government Current Expenditure

In [ ]:
fgc_expenditure = pd.read_csv("/content/Project_data/FGC_Expenditure.csv")

In [ ]:
fgc_expenditure.head()

,DATE,FGEXPND
0,1/1/1947,40.902
1,4/1/1947,40.346
2,7/1/1947,42.007
3,10/1/1947,38.149
4,1/1/1948,38.915


In [ ]:
fgc_expenditure['DATE'] = pd.to_datetime(fgc_expenditure ['DATE']).dt.to_period('M')
fgc_expenditure = fgc_expenditure.set_index('DATE').resample('M').interpolate()

In [ ]:
fgc_expenditure

,FGEXPND
DATE,
1947-01,40.902000
1947-02,40.716667
1947-03,40.531333
1947-04,40.346000
1947-05,40.899667
...,...
2023-06,6382.392667
2023-07,6400.442000
2023-08,6422.158333


In [ ]:
fgc_expenditure.to_csv('fgc_exp.csv')

In [ ]:
fgc_exp = pd.read_csv("/content/fgc_exp.csv")

In [ ]:
fgc_exp

,DATE,FGEXPND
0,1947-01,40.902000
1,1947-02,40.716667
2,1947-03,40.531333
3,1947-04,40.346000
4,1947-05,40.899667
...,...,...
917,2023-06,6382.392667
918,2023-07,6400.442000
919,2023-08,6422.158333
920,2023-09,6443.874667


### Quarterly unemployment Rate (obtained from Fred)

In [ ]:
unemployment_rate = pd.read_csv("/content/Project_data/Unemployment_rate.csv")

In [ ]:
unemployment_rate

,DATE,LRUNTTTTZAQ156S
0,7/1/2000,23.274040
1,10/1/2000,23.948470
2,1/1/2001,24.622900
3,4/1/2001,25.390175
4,7/1/2001,26.157450
...,...,...
89,10/1/2022,32.735710
90,1/1/2023,32.591560
91,4/1/2023,32.914260
92,7/1/2023,31.881060


In [ ]:
unemployment_rate['DATE'] = pd.to_datetime(unemployment_rate['DATE']).dt.to_period('M')
unemployment_rate.set_index('DATE').resample('M').interpolate()

,LRUNTTTTZAQ156S
DATE,
2000-07,23.274040
2000-08,23.498850
2000-09,23.723660
2000-10,23.948470
2000-11,24.173280
...,...
2023-06,32.225460
2023-07,31.881060
2023-08,31.960503


In [ ]:
unemployment_rate

,DATE,LRUNTTTTZAQ156S
0,2000-07,23.274040
1,2000-10,23.948470
2,2001-01,24.622900
3,2001-04,25.390175
4,2001-07,26.157450
...,...,...
89,2022-10,32.735710
90,2023-01,32.591560
91,2023-04,32.914260
92,2023-07,31.881060


In [ ]:
unemployment_rate.rename(columns={'LRUNTTTTZAQ156S': 'UNEPLT'}, inplace=True)

In [ ]:
unemployment_rate.to_csv('uneplt_rate.csv', index=False)

In [ ]:
pd.read_csv("/content/uneplt_rate.csv")

,DATE,UNEPLT
0,2000-07,23.274040
1,2000-10,23.948470
2,2001-01,24.622900
3,2001-04,25.390175
4,2001-07,26.157450
...,...,...
89,2022-10,32.735710
90,2023-01,32.591560
91,2023-04,32.914260
92,2023-07,31.881060


### Money supply (M3)

In [ ]:
Money_supply_M3 = pd.read_csv("/content/Project_data/Money_supply_M3.csv")

In [ ]:
Money_supply_M3.head()

,DATE,MABMM301ZAM189S
0,1965-03-01,4.796183e+09
1,1965-04-01,4.861149e+09
2,1965-05-01,4.862665e+09
3,1965-06-01,4.813954e+09
4,1965-07-01,4.849492e+09


In [ ]:
Money_supply_M3["DATE"] = pd.to_datetime(Money_supply_M3["DATE"])
Money_supply_M3.set_index('DATE', inplace=True)

In [ ]:
Money_supply_M3.to_csv("M3.csv")

### Converting the date format from '%Y-%m-%D' to '%Y-%m'

In [ ]:
Money_supply_M3 = pd.read_csv("/content/M3.csv")

In [ ]:
Money_supply_M3['DATE'] = pd.to_datetime(Money_supply_M3['DATE']).dt.to_period('M')

In [ ]:
Money_supply_M3.rename(columns={'MABMM301ZAM189S': 'M3'}, inplace=True)

In [ ]:
Money_supply_M3.to_csv("M3.csv", index=False)

In [ ]:
M3 = pd.read_csv("/content/M3.csv")

In [ ]:
M3

,DATE,M3
0,1965-03,4.796183e+09
1,1965-04,4.861149e+09
2,1965-05,4.862665e+09
3,1965-06,4.813954e+09
4,1965-07,4.849492e+09
...,...,...
700,2023-07,4.967592e+12
701,2023-08,4.978144e+12
702,2023-09,4.997624e+12
703,2023-10,4.967379e+12


### Annual unemployment data (Converting from annual to monthly)

In [ ]:
uneplt_rate_2 = pd.read_csv("/content/Project_data/Unemployment_rate_2.csv")

In [ ]:
uneplt_rate_2.head()

,DATE,UNEPLT
0,1991,20.543
1,1992,20.945
2,1993,20.819
3,1994,20.464
4,1995,20.032


In [ ]:
uneplt_rate_2['DATE'] = pd.to_datetime(uneplt_rate_2['DATE'], format='%Y')

In [ ]:
uneplt_rate_2 = uneplt_rate_2.set_index('DATE').resample('M').ffill().reset_index()

In [ ]:
uneplt_rate_2

,DATE,UNEPLT
0,1991-01-31,20.543
1,1991-02-28,20.543
2,1991-03-31,20.543
3,1991-04-30,20.543
4,1991-05-31,20.543
...,...,...
380,2022-09-30,28.840
381,2022-10-31,28.840
382,2022-11-30,28.840
383,2022-12-31,28.840


In [ ]:
uneplt_rate_2.to_csv('uneplt_rate.csv', index=False)

In [ ]:
uneplt = pd.read_csv("/content/uneplt_rate.csv")

In [ ]:
uneplt

,DATE,UNEPLT
0,1991-01-31,20.543
1,1991-02-28,20.543
2,1991-03-31,20.543
3,1991-04-30,20.543
4,1991-05-31,20.543
...,...,...
380,2022-09-30,28.840
381,2022-10-31,28.840
382,2022-11-30,28.840
383,2022-12-31,28.840


### Converting the date format from '%Y-%m-%D' to '%Y-%m'

In [ ]:
uneplt['DATE'] = pd.to_datetime(uneplt['DATE']).dt.to_period('M')

In [ ]:
uneplt

,DATE,UNEPLT
0,1991-01,20.543
1,1991-02,20.543
2,1991-03,20.543
3,1991-04,20.543
4,1991-05,20.543
...,...,...
380,2022-09,28.840
381,2022-10,28.840
382,2022-11,28.840
383,2022-12,28.840


In [ ]:
uneplt.to_csv("Uneplt.csv", index=False)

In [ ]:
uneplt = pd.read_csv("/content/Uneplt.csv")

In [ ]:
uneplt

,DATE,UNEPLT
0,1991-01,20.543
1,1991-02,20.543
2,1991-03,20.543
3,1991-04,20.543
4,1991-05,20.543
...,...,...
380,2022-09,28.840
381,2022-10,28.840
382,2022-11,28.840
383,2022-12,28.840


### Consumer Price Index (CPI)

In [ ]:
CPI = pd.read_csv("/content/Project_data/CPI.csv")

In [ ]:
CPI.head()

,DATE,ZAFCPIALLMINMEI
0,1/1/1960,1.345732
1,2/1/1960,1.345732
2,3/1/1960,1.350248
3,4/1/1960,1.354764
4,5/1/1960,1.359279


In [ ]:
CPI['DATE'] = pd.to_datetime(CPI['DATE']).dt.to_period('M')

In [ ]:
CPI

,DATE,ZAFCPIALLMINMEI
0,1960-01,1.345732
1,1960-02,1.345732
2,1960-03,1.350248
3,1960-04,1.354764
4,1960-05,1.359279
...,...,...
762,2023-07,150.016879
763,2023-08,150.421965
764,2023-09,151.502194
765,2023-10,152.852481


In [ ]:
CPI.rename(columns={'ZAFCPIALLMINMEI': 'CPI'}, inplace=True)

In [ ]:
CPI.to_csv("CPI.csv", index=False)

In [ ]:
CPI = pd.read_csv("/content/CPI.csv")

In [ ]:
CPI

,DATE,CPI
0,1960-01,1.345732
1,1960-02,1.345732
2,1960-03,1.350248
3,1960-04,1.354764
4,1960-05,1.359279
...,...,...
762,2023-07,150.016879
763,2023-08,150.421965
764,2023-09,151.502194
765,2023-10,152.852481


### Merging the CSV files together

In [ ]:
mergedData = pd.merge(ex_rate, int_rate, on='DATE', how='inner')

In [ ]:
mergedData = pd.merge(mergedData, PPI_Industrial, on='DATE', how='inner')

In [ ]:
mergedData = pd.merge(mergedData, PPI_Manufacturing, on='DATE', how='inner')

In [ ]:
mergedData = pd.merge(mergedData, Real_GDP, on='DATE', how='inner')

In [ ]:
mergedData = pd.merge(mergedData, RS_volume, on='DATE', how='inner')

In [ ]:
mergedData = pd.merge(mergedData, trade_balance, on='DATE', how='inner')

In [ ]:
mergedData = pd.merge(mergedData, fgc_exp, on='DATE', how='inner')

In [ ]:
mergedData = pd.merge(mergedData, uneplt, on='DATE', how='inner')

In [ ]:
mergedData = pd.merge(mergedData, M3, on='DATE', how='inner')

In [ ]:
mergedData = pd.merge(mergedData, CPI, on='DATE', how='inner')

In [ ]:
mergedData.to_csv("mergedData.csv", index=False)